In [1]:
import pandas as pd
from copy import deepcopy

# PAM - Getting Started

**Pandemic Activity Modelling/Modifying**

This notebook is a quick spin through PAM. Intended to provide a technical overview and inspire further research and development.

### Load Data

Here we load simple travel diary data of London commuters. This is a very simple 0.1% sample of data about work and education commutes from the 2011 census. Because we're sharing this date - we've aggregated locations to borough level and randomized personal attributes - so don't get too excited about the results.

The data is available in `pam/example_data`.

In [2]:
trips = pd.read_csv('/Users/fred.shone/Projects/pam/example_data/example_travel_diaries.csv')
attributes = pd.read_csv('/Users/fred.shone/Projects/pam/example_data/example_attributes.csv')
attributes.set_index('pid', inplace=True)

In [3]:
trips.head()

,uid,pid,hid,seq,hzone,ozone,dzone,purp,mode,tst,tet,freq
0,0,census_0,census_0,0,Harrow,Harrow,Camden,work,pt,444,473,1000
1,1,census_0,census_0,1,Harrow,Camden,Harrow,work,pt,890,919,1000
2,2,census_1,census_1,0,Greenwich,Greenwich,Tower Hamlets,work,pt,507,528,1000
3,3,census_1,census_1,1,Greenwich,Tower Hamlets,Greenwich,work,pt,1065,1086,1000
4,4,census_2,census_2,0,Croydon,Croydon,Croydon,work,pt,422,425,1000


### Build Activity Plans

First we convert the travel diary data to Activity Plans:

In [4]:
from pam import parse
population = parse.load_travel_diary(trips, attributes)

Let's check out an example Activity Plan and Attributes:

In [5]:
household = population.households['census_123']
person = household.people['census_123']
person.print_plan()

0:	Activity(act:home, area:Hillingdon, time:00:00:00 --> 07:13:00, duration:7:13:00)
1:	Leg(mode:pt, area:Hillingdon --> Ealing, time:07:13:00 --> 07:38:00, duration:0:25:00)
2:	Activity(act:work, area:Ealing, time:07:38:00 --> 16:58:00, duration:9:20:00)
3:	Leg(mode:pt, area:Ealing --> Hillingdon, time:16:58:00 --> 17:23:00, duration:0:25:00)
4:	Activity(act:home, area:Hillingdon, time:17:23:00 --> 23:59:00, duration:6:36:00)


In [6]:
person.attributes

{'gender': 'male', 'job': 'work', 'occ': 'blue', 'inc': 'high'}

Before we do any activity modification - we create a simple function to extract some example statistics. We include this as a simple demo, but would love to add more.

Note that activity plans allow us to consider detailed **joint** segmentations, such as socio-economic, spatial, temporal, modal, activity sequence and so on.

In [7]:
def print_simple_stats(population):
    """
    Print some simple population statistics.
    """
    time_at_home = 0
    travel_time = 0 
    low_income_central_trips = 0
    high_income_central_trips = 0
    
    for hid, hh in population.households.items():
        for pid, person in hh.people.items():
            freq = person.freq
            
            for p in person.plan:
                if p.act == 'travel':
                    duration = p.duration.seconds * freq / 3600
                    travel_time += duration
                    
                    if p.end_area == "Westminster,City of London":
                        if person.attributes['inc'] == "low":
                            low_income_central_trips += freq
                            
                        elif person.attributes['inc'] == "high":
                            high_income_central_trips += freq
                    
                else:  # activity
                    if p.act == 'home':
                        duration = p.duration.seconds * freq / 3600
                        time_at_home += duration
                        
    print(f"Population total time at home: {time_at_home/1000000:.2f} million hours")
    print(f"Population total travel time: {travel_time/1000000:.2f} million hours")
    print(f"Low income trips to Central London: {low_income_central_trips} trips")
    print(f"High income trips to Central London: {high_income_central_trips} trips")
                        

In [8]:
print_simple_stats(population)

Population total time at home: 49.46 million hours
Population total travel time: 1.58 million hours
Low income trips to Central London: 229000 trips
High income trips to Central London: 246000 trips


### Scenarios

Our 2011 baseline London population of commuters seems sensible, they spend about 50 million hours at home and 1.6 million hours travelling.

But what if we want to try and build some more up to date scenarios?

We consider 3 modifiers:

1. A household will be quarantined with p=0.025
2. A person will be staying at home (self isolating) with p=0.1
3. Education activities will be removed and plans adjusted with p=0.9

In [9]:
from pam import modify
policy1 = modify.HouseholdQuarantined(probability=0.025)
policy2 = modify.PersonStayAtHome(probability=0.1)
policy3 = modify.RemoveActivity(['education'], probability=0.9)

We then build 3 example population scenarios:

1. policy 1
2. policy 1 + policy 2
3. policy 1 + policy 2 + policy 3

In [10]:
def apply_policies(population, policies: list):

    new_population = deepcopy(population) 
    for hid, household in new_population.households.items():
        for policy in policies:
            policy.apply_to(household)
    return new_population

In [11]:
population1 = apply_policies(population, [policy1])
population2 = apply_policies(population, [policy1, policy2])
population3 = apply_policies(population, [policy1, policy2, policy3])

### Review

So this has been a pretty quick and dirty demo. But we can still take a look at our new population scenario stats below. Obviously we'd like to add some more analytics... some viz... some validation... then output new travel plans and OD matrices for use in other models.

In [12]:
print_simple_stats(population1)

Population total time at home: 50.05 million hours
Population total travel time: 1.54 million hours
Low income trips to Central London: 222000 trips
High income trips to Central London: 239000 trips


In [13]:
print_simple_stats(population2)

Population total time at home: 51.97 million hours
Population total travel time: 1.39 million hours
Low income trips to Central London: 205000 trips
High income trips to Central London: 204000 trips


In [14]:
print_simple_stats(population3)

Population total time at home: 56.28 million hours
Population total travel time: 1.14 million hours
Low income trips to Central London: 171000 trips
High income trips to Central London: 180000 trips


### Footnote - Build Your Own Person

In [15]:
from pam.core import Population, Household, Person
from pam.activity import Plan, Activity, Leg
from pam.utils import minutes_to_datetime

In [16]:
person = Person(1)
person.add(
    Activity(
        seq=1,
        act='home',
        area='a',
        start_time=minutes_to_datetime(0),
        end_time=minutes_to_datetime(60)
    )
)
person.add(
    Leg(
        seq=1,
        mode='car',
        start_area='a',
        end_area='b',
        start_time=minutes_to_datetime(60),
        end_time=minutes_to_datetime(90)
    )
)
person.add(
    Activity(
        seq=2,
        act='education',
        area='b',
        start_time=minutes_to_datetime(90),
        end_time=minutes_to_datetime(120)
    )
)
person.add(
    Leg(
        seq=2,
        mode='car',
        start_area='b',
        end_area='a',
        start_time=minutes_to_datetime(120),
        end_time=minutes_to_datetime(180)
    )
)
person.add(
    Activity(
        seq=3,
        act='home',
        area='a',
        start_time=minutes_to_datetime(180),
        end_time=minutes_to_datetime(24 * 60 - 1)
    )
)

In [17]:
person.print_plan()

0:	Activity(act:home, area:a, time:00:00:00 --> 01:00:00, duration:1:00:00)
1:	Leg(mode:car, area:a --> b, time:01:00:00 --> 01:30:00, duration:0:30:00)
2:	Activity(act:education, area:b, time:01:30:00 --> 02:00:00, duration:0:30:00)
3:	Leg(mode:car, area:b --> a, time:02:00:00 --> 03:00:00, duration:1:00:00)
4:	Activity(act:home, area:a, time:03:00:00 --> 23:59:00, duration:20:59:00)


In [18]:
a, b = person.remove_activity(2)

In [19]:
person.print_plan()

0:	Activity(act:home, area:a, time:00:00:00 --> 01:00:00, duration:1:00:00)
1:	Leg(mode:car, area:a --> b, time:01:00:00 --> 01:30:00, duration:0:30:00)
2:	Leg(mode:car, area:b --> a, time:02:00:00 --> 03:00:00, duration:1:00:00)
3:	Activity(act:home, area:a, time:03:00:00 --> 23:59:00, duration:20:59:00)


In [20]:
person.fill_plan(a,b)

True

In [21]:
person.print_plan()

0:	Activity(act:home, area:a, time:00:00:00 --> 23:59:00, duration:23:59:00)
